### Applied Data Science Capstone Week 3 - 3

In [1]:
from bs4 import BeautifulSoup
import pandas as pd

import numpy as np # library to handle data in a vectorized manner


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


Download data.

In [2]:
!wget -q -O 'List_of_postal_codes_of_Canada.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
# postalcode - geographical coordinates
!wget -q -O 'geopostal.csv' https://cocl.us/Geospatial_data
    
!wget -q -O 'tront.geojson' https://geo.nyu.edu/download/file/nyu-2451-36180-geojson.json

Read a downloaded geojson.

In [3]:
with open('tront.geojson') as json_data:
    tront_geojson = json.load(json_data)
#newyork_data

Read rows in file that was downloaded from http://cocl.us/Geospatial_data except for the top row.

In [4]:
import csv
postal_to_geographica_coordinates = {}
with open('geopostal.csv') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    
    for index, row in enumerate(csvreader):
        if index > 0 :  postal_to_geographica_coordinates.setdefault(row[0], (row[1],row[2]))
#postal_to_geographica_coordinates            

Function tp get text in table's cell is in &lt;td&gt;abc&lt;/td&gt; or &lt;td&gt;&lt;a&gt;abc&lt;/a&gt;&lt;/td&gt;.

In [5]:
def get_text_in_tag(tag):
    """
    get text in like <td>aaa</td> or <td><a>aaa</a></td> from argument tag.
    """
    if tag.string is not None:
        return tag.string
    else:
        return tag.find('a').string

                             

Load a downloaded wiki page.

In [6]:
with open('List_of_postal_codes_of_Canada.html') as html_file:
    soup = BeautifulSoup(html_file,"lxml")

Read the 2nd or later rows in table and create dictionary.<br/>
When postalcode as key is in dictionary alaready, add Neighbourhood to array in value of dictionary.

In [7]:
not_assigned = 'Not assigned'
table_header=[ 'Neighborhood', 'PostalCode', 'Borough', 'Latitude', 'Longitude', 'IsTront']

table_values = []
postcode_dict = {} 

table_tag = soup.table
first_row = table_tag.find('tr')

for tr in first_row.find_next_siblings():
    row = [ get_text_in_tag(td) if td.string is None else td.string.strip() for td in tr.find_all('td')]

    if row[1] == not_assigned:
        continue
    elif row[2] == not_assigned:
        postcode_dict.setdefault(row[1],  [row[0], row[1], postal_to_geographica_coordinates[row[0]], row[1].find('Tront') > -1  ])
    else:
        postcode_dict.setdefault(row[2],  [row[0], row[1], postal_to_geographica_coordinates[row[0]], row[1].find('Toronto') > -1  ])
        
data_src = [ [key, value[0], value[1], float(value[2][0]), float(value[2][1]), value[3] ] for key,value in postcode_dict.items()]     

Create a DataFrame.

In [8]:
neighborhoods = pd.DataFrame(data_src, columns=table_header)
print(neighborhoods.shape)
neighborhoods.head()

(210, 6)


,Neighborhood,PostalCode,Borough,Latitude,Longitude,IsTront
0,Fairview,M2J,North York,43.778517,-79.346556,False
1,North Midtown,M5R,Central Toronto,43.672710,-79.405678,True
2,Martin Grove Gardens,M9R,Etobicoke,43.688905,-79.554724,False
3,Downsview North,M3H,North York,43.754328,-79.442259,False
4,Old Mill North,M8X,Etobicoke,43.653654,-79.506944,False


Get the geograpical coordinate of Tronto city. 

In [9]:
address = 'Toronto city'

geolocator = Nominatim()
location = geolocator.geocode(address)
toronto_latitude = location.latitude
toronto_longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(toronto_latitude, toronto_longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [10]:
toronto_data = neighborhoods.loc[neighborhoods['IsTront'] == True]
toronto_data.head()

,Neighborhood,PostalCode,Borough,Latitude,Longitude,IsTront
1,North Midtown,M5R,Central Toronto,43.672710,-79.405678,True
7,Little Portugal,M6J,West Toronto,43.647927,-79.419750,True
9,Summerhill West,M4V,Central Toronto,43.686412,-79.400049,True
11,Riverdale,M4K,East Toronto,43.679557,-79.352188,True
12,Richmond,M5H,Downtown Toronto,43.650571,-79.384568,True


Show neighborhoods of Toronto city

In [11]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = toronto_data['PostalCode']
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [12]:
# @hidden_cell
CLIENT_ID = 'SCXVI3PF4XNWFUT0FM54TC0GTSS1JHEQ3PT500M421AGHWH5' # your Foursquare ID
CLIENT_SECRET = 'RVTD4SO1HUNBLGNWN04KSU1WKRTDXMB41ELHKT5V4YDKMPEO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT=100

The function to repeat the same process to all the neighborhood

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get venues in Tronto via foursquare API.

In [14]:
toronto_venues = getNearbyVenues( names = toronto_data['Neighborhood'],
                                   latitudes = toronto_data['Latitude'],
                                   longitudes = toronto_data['Longitude']
                                  )

North Midtown
Little Portugal
Summerhill West
Riverdale
Richmond
Roncesvalles
Forest Hill SE
South Niagara
Parkdale Village
Church and Wellesley
Island airport
The Junction South
The Beaches West
Railway Lands
Union Station
Adelaide
Grange Park
University of Toronto
Dufferin
Davisville
Parkdale
King and Spadina
Garden District
North Toronto West
Forest Hill North
Toronto Dominion Centre
Lawrence Park
Harbourfront
Stn A PO Boxes 25 The Esplanade
Bathurst Quay
Ryerson
Regent Park
Chinatown
Roselawn
Exhibition Place
Toronto Islands
Commerce Court
The Annex
South Hill
First Canadian Place
Davisville North
The Danforth West
Underground city
Brockton
Business reply mail Processing Centre969 Eastern
CN Tower
High Park
India Bazaar
Berczy Park
Design Exchange
Victoria Hotel
The Beaches
Harbourfront West
Cabbagetown
Swansea
Rosedale
Dovercourt Village
Harbourfront East
Studio District
Moore Park
Forest Hill West
Kensington Market
Deer Park
Christie
Summerhill East
King
Rathnelly
Yorkville
Trini

In [15]:
print(toronto_venues.shape)
toronto_venues.head()

(3175, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North Midtown,43.67271,-79.405678,Ezra's Pound,43.675153,-79.405858,Café
1,North Midtown,43.67271,-79.405678,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
2,North Midtown,43.67271,-79.405678,Bar Begonia,43.675093,-79.406406,French Restaurant
3,North Midtown,43.67271,-79.405678,Rose & Sons,43.675668,-79.403617,American Restaurant
4,North Midtown,43.67271,-79.405678,Live Organic Food Bar,43.675053,-79.406715,Vegetarian / Vegan Restaurant


Check how many venues were returned for each neighborhood

In [16]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,13,13,13,13,13,13
Berczy Park,55,55,55,55,55,55
Brockton,19,19,19,19,19,19
Business reply mail Processing Centre969 Eastern,18,18,18,18,18,18
CN Tower,13,13,13,13,13,13
Cabbagetown,46,46,46,46,46,46
Central Bay Street,78,78,78,78,78,78
Chinatown,97,97,97,97,97,97


Let's find out how many unique categories can be curated from all the returned venues

In [17]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 231 uniques categories.


Analyze Each Neighborhood

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,North Midtown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

examine the new dataframe size.

In [19]:
toronto_onehot.shape

(3175, 231)

group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.000000,0.010000,0.010000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.01,0.020000,0.000000,0.000000,0.020000,0.010000,0.00,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.010000,0.060000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.030000,0.000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.010000,0.00

show shape

In [21]:
toronto_grouped.shape

(72, 231)

print each neighborhood along with the top 5 most common venues

In [22]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----Bathurst Quay----
              venue  freq
0    Airport Lounge  0.15
1   Airport Service  0.15
2  Airport Terminal  0.15
3  Sculpture Garden  0.08
4           Airport  0.08


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2          Restaurant  0.05
3  Italian Restaurant  0.04
4      Farmers Market  0.04


----Brockton----
            venue  freq
0  Breakfast Spot  0.11
1     Coffee Shop  0.11
2            Café  0.11
3     Yoga Studio  0.05
4             Gym  0.05


----Business reply mail Processing Centre969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2    Recording Studio  0.06
3          Skate Park  0.06
4                 Spa  0.06


----CN Tower----
              venue  freq
0   

Write a function to sort the venues in descending order.

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

reate the new dataframe and display the top 10 venues for each neighborhood.

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Gym,Bar,Hotel,Cosmetics Shop,Restaurant
1,Bathurst Quay,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boat or Ferry,Sculpture Garden,Boutique,Airport Gate,Airport,Airport Food Court
2,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Pub,Café
3,Brockton,Coffee Shop,Café,Breakfast Spot,Yoga Studio,Italian Restaurant,Performing Arts Venue,Pet Store,Climbing Gym,Caribbean Restaurant,Burrito Place
4,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop
5,CN Tower,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boat or Ferry,Sculpture Garden,Boutique,Airport Gate,Airport,Airport Food Court
6,Cabbagetown,Coffee Shop,Restaurant,Pizza Place,Italian Restaurant,Indian Restaurant,Bakery,Café,Pub,Japanese Restaurant,Diner
7,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Bar,Ice Cream Shop,Burger Joint,Thai Restaurant,Chinese Restaurant,Spa
8,Chinatown,Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Bakery,Mexican Restaurant,Chinese Restaurant,Dim Sum Restaurant,Dessert Shop
9,Christie,Café,Grocery Store,Park,Italian Restaurant,Restaurant,Diner,Nightclub,Baby Store,Coffee Shop,Convenience Store


# Crustering

Run k-means to cluster the neighborhood into 5 clusters.

In [25]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [26]:
toronto_merged = toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Neighborhood,PostalCode,Borough,Latitude,Longitude,IsTront,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North Midtown,M5R,Central Toronto,43.672710,-79.405678,True,1,Coffee Shop,Café,Sandwich Place,Pizza Place,Jewish Restaurant,French Restaurant,BBQ Joint,Pub,Indian Restaurant,American Restaurant
7,Little Portugal,M6J,West Toronto,43.647927,-79.419750,True,2,Bar,Men's Store,Restaurant,Café,Coffee Shop,Asian Restaurant,Vietnamese Restaurant,Cocktail Bar,Pizza Place,Ice Cream Shop
9,Summerhill West,M4V,Central Toronto,43.686412,-79.400049,True,1,Coffee Shop,Pub,American Restaurant,Sushi Restaurant,Fried Chicken Joint,Bagel Shop,Sports Bar,Supermarket,Pizza Place,Light Rail Station
11,Riverdale,M4K,East Toronto,43.679557,-79.352188,True,1,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Health Food Store,Furniture / Home Store,Pub,Pizza Place,Liquor Store
12,Richmond,M5H,Downtown Toronto,43.650571,-79.384568,True,1,Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Gym,Bar,Hotel,Cosmetics Shop,Restaurant


 visualize the resulting clusters

In [27]:
# create map
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

Cluster 1

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
123,The Annex,Coffee Shop,Café,Sandwich Place,Pizza Place,Jewish Restaurant,French Restaurant,BBQ Joint,Pub,Indian Restaurant,American Restaurant
161,Dovercourt Village,Pharmacy,Supermarket,Bakery,Liquor Store,Fast Food Restaurant,Middle Eastern Restaurant,Discount Store,Music Venue,Park,Pool


Cluster 2

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North Midtown,Coffee Shop,Café,Sandwich Place,Pizza Place,Jewish Restaurant,French Restaurant,BBQ Joint,Pub,Indian Restaurant,American Restaurant
9,Summerhill West,Coffee Shop,Pub,American Restaurant,Sushi Restaurant,Fried Chicken Joint,Bagel Shop,Sports Bar,Supermarket,Pizza Place,Light Rail Station
11,Riverdale,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Health Food Store,Furniture / Home Store,Pub,Pizza Place,Liquor Store
12,Richmond,Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Gym,Bar,Hotel,Cosmetics Shop,Restaurant
16,Forest Hill SE,Coffee Shop,Pub,American Restaurant,Sushi Restaurant,Fried Chicken Joint,Bagel Shop,Sports Bar,Supermarket,Pizza Place,Light Rail Station
17,South Niagara,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boat or Ferry,Sculpture Garden,Boutique,Airport Gate,Airport,Airport Food Court
18,Parkdale Village,Coffee Shop,Café,Breakfast Spot,Yoga Studio,Italian Restaurant,Performing Arts Venue,Pet Store,Climbing Gym,Caribbean Restaurant,Burrito Place
19,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Restaurant,Burger Joint,Men's Store,Fast Food Restaurant,Bubble Tea Shop,Café
23,Island airport,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boat or Ferry,Sculpture Garden,Boutique,Airport Gate,Airport,Airport Food Court
24,The Junction South,Café,Mexican Restaurant,Bar,Grocery Store,Arts & Crafts Store,Music Venue,Diner,Cajun / Creole Restaurant,Sandwich Place,Fast Food Restaurant


Cluster 3

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Little Portugal,Bar,Men's Store,Restaurant,Café,Coffee Shop,Asian Restaurant,Vietnamese Restaurant,Cocktail Bar,Pizza Place,Ice Cream Shop
15,Roncesvalles,Breakfast Spot,Gift Shop,Dessert Shop,Dog Run,Restaurant,Cuban Restaurant,Movie Theater,Bookstore,Coffee Shop,Burger Joint
92,Stn A PO Boxes 25 The Esplanade,Coffee Shop,Restaurant,Seafood Restaurant,Café,Cocktail Bar,Pub,Hotel,Italian Restaurant,Beer Bar,Cosmetics Shop
97,Regent Park,Coffee Shop,Café,Park,Bakery,Pub,Breakfast Spot,Theater,Mexican Restaurant,Health Food Store,Shoe Store
109,Exhibition Place,Coffee Shop,Café,Breakfast Spot,Yoga Studio,Italian Restaurant,Performing Arts Venue,Pet Store,Climbing Gym,Caribbean Restaurant,Burrito Place
129,Underground city,Café,Coffee Shop,Hotel,Restaurant,American Restaurant,Seafood Restaurant,Bar,Bakery,Asian Restaurant,Gym
154,Harbourfront West,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boat or Ferry,Sculpture Garden,Boutique,Airport Gate,Airport,Airport Food Court


Cluster 4

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,Union Station,Coffee Shop,Aquarium,Hotel,Café,Pizza Place,Italian Restaurant,Brewery,Restaurant,Scenic Lookout,Bakery
78,King and Spadina,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boat or Ferry,Sculpture Garden,Boutique,Airport Gate,Airport,Airport Food Court
130,Brockton,Coffee Shop,Café,Breakfast Spot,Yoga Studio,Italian Restaurant,Performing Arts Venue,Pet Store,Climbing Gym,Caribbean Restaurant,Burrito Place
153,The Beaches,Playground,Coffee Shop,Pub,Women's Store,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
162,Harbourfront East,Coffee Shop,Aquarium,Hotel,Café,Pizza Place,Italian Restaurant,Brewery,Restaurant,Scenic Lookout,Bakery
167,Forest Hill West,Trail,Jewelry Store,Sushi Restaurant,Bus Line,Women's Store,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


Cluster 5

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
145,Design Exchange,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Deli / Bodega,Gym,Gastropub,Bar,Seafood Restaurant


end